In [48]:
import pandas as pd
import math
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import warnings 
from datetime import date

from scipy import stats

from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns 


from sklearn.feature_selection import f_classif # anova
from sklearn.feature_selection import chi2 # хи-квадрат
from sklearn.feature_selection import SelectPercentile

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor 
# инструменты для оценки точности модели
from sklearn import metrics 

In [49]:
df = pd.read_csv('./clean_data.csv')

In [50]:
df.columns

Index(['price_b', 'token_age_b', 'transfers', 'buys_m5', 'sells_m5', 'buys_h1',
       'sells_h1', 'volume_m5', 'volume_h1', 'price_change_m5',
       'price_change_h1', 'price_change_h6', 'price_change_h24', 'liquidity',
       'market_cap', 'is_mutable_metadata', 'is_telegram', 'is_twitter',
       'is_website', 'boosts', 'telegram_members', 'twitter_days',
       'twitter_tweets', 'twitter_followers', 'twitter_smart_followers',
       'closing_date', 'sns_held_all', 'sns_sold_some', 'sns_sold_all', 'PNL',
       'PNL_10', 'PNL_20', 'PNL_30', 'PNL_40', 'PNL_50', 'price_change_check',
       'sns_most_common', 'sns_most_common_except_0', 'tt_f10_bought',
       'tt_f10_sold', 'sns_f10_bought', 'sns_f10_sold', 'sns_no_bought',
       'sns_no_sold', 'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss',
       'tt_pnl_loss', 'profit'],
      dtype='object')

In [51]:
x = df.drop(columns=[
    # 'Unnamed: 53',
    #    'Unnamed: 54',
    
    "profit", "transfers",
    'price_change_h6', 'price_change_h24',
    'volume_h1',
    'price_change_h1',
    'buys_h1', 'sells_h1',
    'PNL', 'PNL_10', 'PNL_20', 'PNL_30', 'PNL_40', 'PNL_50', 'closing_date'
    # 'telegram_members', 'twitter_days', 
    # 'twitter_tweets', 'twitter_followers', 'twitter_smart_followers', 
    # 'sns_most_common', 'sns_most_common_except_0',
    #    'tt_f10_bought', 'tt_f10_sold', 'sns_f10_bought', 'sns_no_bought',
    #    'sns_no_sold', 'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss',
    #    'tt_pnl_loss'
    ])
y = df["profit"].copy()

In [52]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   price_b                   145 non-null    float64
 1   token_age_b               145 non-null    float64
 2   buys_m5                   145 non-null    float64
 3   sells_m5                  145 non-null    float64
 4   volume_m5                 145 non-null    float64
 5   price_change_m5           145 non-null    float64
 6   liquidity                 145 non-null    float64
 7   market_cap                145 non-null    float64
 8   is_mutable_metadata       145 non-null    int64  
 9   is_telegram               145 non-null    int64  
 10  is_twitter                145 non-null    int64  
 11  is_website                145 non-null    int64  
 12  boosts                    145 non-null    int64  
 13  telegram_members          145 non-null    int64  
 14  twitter_da

In [53]:
# Объявляем вспомогательную функцию по определению наличия выбросов в признаке
def there_are_outliers(data, feature) -> bool:
    '''
    Функция определяющая методом Тьюки наличие выбросов в признаке
    Args:
        data (DataFrame): датасет
        feature (str): наименование признака
    Returns:
        boolean: возвращает True если выбросы в признаке есть в наличии 
    '''
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return x.min() < lower_bound or x.max() > upper_bound

In [54]:
warnings.simplefilter("ignore", UserWarning)

In [55]:
alpha = 0.05 

scale_columns = [
    'token_age_b', 
    #'transfers', 'transactions', 
    'buys_m5', 'sells_m5',
       #'buys_h1', 'sells_h1', 
       'volume_m5', 
       #'volume_h1', 
       'price_change_m5',
       #'price_change_h1', 
    #    'price_change_h6', 'price_change_h24', 
       'liquidity',
       'market_cap', 
       'boosts', 
       'telegram_members', 'twitter_days',
       'twitter_tweets', 'twitter_followers', 'twitter_smart_followers',
       'sns_held_all', 'sns_sold_some', 'sns_sold_all',
       'tt_f10_bought', 'tt_f10_sold', 'sns_f10_bought', 'sns_f10_sold', 'sns_no_bought',
       'sns_no_sold', 'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss',
       'tt_pnl_loss', 'sns_most_common', 'price_change_check'
]
s = []
# Приступаем к маштабированию признаков,
# исходные признаки не трогаем, создаем отдельные,
# т.к. код в данной ячейке может быть исполнен повторно
for feature in scale_columns:
    # С помощью теста Шапиро-Уилка проверим, нормально ли распределён признак 
    result = stats.shapiro(x[feature])
    if result[1] <= alpha:
        print('Признак {} распределен не нормально,'.format(feature))
        # С помощью метода Тьюки проверим, есть ли выбросы
        if there_are_outliers(x, feature):
            print('в признаке {} имеются выбросы, используем RobustScaler().'
                  .format(feature))
            r_scaler = preprocessing.RobustScaler()
            x[str(feature) + '_scaled'] = r_scaler.fit_transform(
                x[feature].values.reshape(x.shape[0],1))
        else:
            print('в признаке {} нет выбросов, используем MinMaxScaler().'
                  .format(feature))
            mm_scaler = preprocessing.MinMaxScaler()
            x[str(feature) + '_scaled'] = mm_scaler.fit_transform(
                x[feature].values.reshape(x.shape[0],1))
    else:
        print('Признак {} распределен нормально,'.format(feature))
        print('используем StandardScaler().')
        s_scaler = preprocessing.StandardScaler()
        x[str(feature) + '_scaled'] = s_scaler.fit_transform(
            x[feature].values.reshape(x.shape[0],1))  
    s.append(str(feature) + '_scaled')



Признак token_age_b распределен не нормально,
в признаке token_age_b имеются выбросы, используем RobustScaler().
Признак buys_m5 распределен не нормально,
в признаке buys_m5 имеются выбросы, используем RobustScaler().
Признак sells_m5 распределен не нормально,
в признаке sells_m5 имеются выбросы, используем RobustScaler().
Признак volume_m5 распределен не нормально,
в признаке volume_m5 имеются выбросы, используем RobustScaler().
Признак price_change_m5 распределен не нормально,
в признаке price_change_m5 имеются выбросы, используем RobustScaler().
Признак liquidity распределен не нормально,
в признаке liquidity имеются выбросы, используем RobustScaler().
Признак market_cap распределен не нормально,
в признаке market_cap имеются выбросы, используем RobustScaler().
Признак boosts распределен не нормально,
в признаке boosts имеются выбросы, используем RobustScaler().
Признак telegram_members распределен не нормально,
в признаке telegram_members имеются выбросы, используем RobustScaler().

In [56]:
x = x.drop(scale_columns, axis=1)

In [57]:
x.head()

,price_b,is_mutable_metadata,is_telegram,is_twitter,is_website,sns_most_common_except_0,token_age_b_scaled,buys_m5_scaled,sells_m5_scaled,volume_m5_scaled,...,sns_f10_bought_scaled,sns_f10_sold_scaled,sns_no_bought_scaled,sns_no_sold_scaled,tt_no_bought_scaled,tt_no_sold_scaled,sns_pnl_loss_scaled,tt_pnl_loss_scaled,sns_most_common_scaled,price_change_check_scaled
0,0.000623,0,0,0,1,0.1,1.583351,-0.190141,-0.156977,-0.095994,...,0.662665,0.755044,1.000000,0.076087,0.31,0.0,-0.428571,0.0,-0.005556,-0.846518
1,0.000280,0,1,1,1,0.1,-0.132267,0.776056,6.372093,3.845939,...,-0.072780,-0.057844,-0.357143,0.282609,0.19,0.0,-0.357143,0.0,0.000000,2.132190
2,0.000412,0,1,1,1,0.1,-0.145185,0.898592,4.575581,0.735190,...,-0.170983,-0.099825,1.071429,0.206522,0.54,0.0,1.571429,0.0,-0.005556,13.003081
3,0.001253,0,0,1,0,43.0,7.872653,-0.295775,-0.563953,-0.444754,...,-0.053115,0.139866,2.000000,0.021739,0.17,0.0,1.857143,0.0,-0.005556,0.843125
4,0.001229,0,1,1,0,0.1,0.597125,-0.150704,0.000000,0.278593,...,0.952525,1.046611,0.857143,0.108696,0.20,0.0,-0.142857,0.0,-0.005556,-0.067286


In [58]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [59]:
x_train = torch.tensor(x_train.values).to(torch.float32)
x_test = torch.tensor(x_test.values).to(torch.float32)
y_train = torch.tensor(y_train.values).to(torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values).to(torch.float32).unsqueeze(1)

In [60]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

In [61]:
n_features = x_train.shape[1]
model = LogisticRegression(n_features)

optim = torch.optim.SGD(model.parameters(), lr=1)
criterion = torch.nn.BCELoss()

In [62]:
EPOCHS = 5

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f'Среднее значение функции потерь за эпоху {e}: {loss.data}')
    return model

In [63]:
model = train(model, optim, criterion, x_train, y_train)

Среднее значение функции потерь за эпоху 1: 12.286874771118164
Среднее значение функции потерь за эпоху 2: 22.421558380126953
Среднее значение функции потерь за эпоху 3: 23.996179580688477
Среднее значение функции потерь за эпоху 4: 22.249683380126953
Среднее значение функции потерь за эпоху 5: 22.320438385009766


In [64]:
def accuracy(model, x, y):
    out = model(x)
    correct = torch.abs(y - out) < 0.5
    return correct.float().mean()

plain_accuracy = accuracy(model, x_test, y_test)
print(f'Точность модели: {plain_accuracy}')

Точность модели: 0.517241358757019


In [65]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

x = df.drop(columns=["profit"])
y = df["profit"].copy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Создание и обучение модели логистической регрессии
model = LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(X_train, y_train)

# Предсказание классов на тестовой выборке
y_pred = model.predict(X_test)

# Вычисление точности классификации
accuracy = accuracy_score(y_test, y_pred)
print("Точность классификации: {:.2f}%".format(accuracy * 100))

ValueError: could not convert string to float: '2025-01-05 07:03:33'

In [66]:
from sklearn.metrics import classification_report  
from sklearn.metrics import accuracy_score  
from sklearn.neighbors import KNeighborsClassifier  

In [67]:
KNN_model = KNeighborsClassifier(n_neighbors=10)

In [68]:
KNN_model.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [69]:
KNN_prediction = KNN_model.predict(x_test)

In [71]:
print(accuracy_score(KNN_prediction, y_test))  
print(classification_report(KNN_prediction, y_test))

0.5862068965517241
              precision    recall  f1-score   support

           0       1.00      0.54      0.70        26
           1       0.20      1.00      0.33         3

    accuracy                           0.59        29
   macro avg       0.60      0.77      0.52        29
weighted avg       0.92      0.59      0.66        29

